<a href="https://colab.research.google.com/github/vchandraiitk/datascience/blob/main/Vikas_M4_NB_MiniProject_01_MLR_MPI_OpenMP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Advanced Certification Program in Computational Data Science
## A program by IISc and TalentSprint
### Mini-Project: Implementation of Multiple Linear Regression using MPI and OpenMP

## Learning Objectives

At the end of the mini-project, you will be able to :

* understand the collective communication operations like scatter, gather, broadcast
* understand the blocking and non-blocking communication
* implement multiple linear regression and run it using MPI
* implement the multiple linear regression based predictions using OpenMP

### Dataset

The dataset chosen for this mini-project is [Combined Cycle Power Plant](https://archive.ics.uci.edu/ml/datasets/combined+cycle+power+plant). The dataset is made up of 9568 records and 5 columns. Each record contains the values for Ambient Temperature, Exhaust Vaccum, Ambient Pressure, Relative Humidity and Energy Output.

Predicting full load electrical power output of a base load power plant is important in order to maximize the profit from the available megawatt hours.  The base load operation of a power plant is influenced by four main parameters, which are used as input variables in the dataset, such as ambient temperature, atmospheric pressure, relative humidity, and exhaust steam pressure. These parameters affect electrical power output, which is considered as the target variable.

**Note:** The data was collected over a six year period (2006-11).

## Information

#### MPI in a Nutshell

MPI stands for "Message Passing Interface". It is a library of functions (in C / Python) or subroutines (in Fortran) that you insert into source code to perform data communication between processes. MPI was developed over two years of discussions led by the MPI Forum, a group of roughly sixty people representing some forty organizations.

To know more about MPI click [here](https://hpc-tutorials.llnl.gov/mpi/)


#### Multiple Linear Regression

Multiple regression is an extension of simple linear regression. It is used when we want to predict the value of a variable based on the value of two or more other variables. The variable we want to predict is called the dependent variable (or sometimes, the outcome, target or criterion variable). The variables we are using to predict the value of the dependent variable are called the independent variables (or sometimes, the predictor, explanatory or regressor variables).

**Note:** We will be using the mpi4py Python package for MPI based code implementation

## Grading = 20 Points

**Run the below code to install mpi4py package**

In [1]:
!pip install mpi4py

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 464.8/464.8 kB 2.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for mpi4py: filename=mpi4py-4.0.0-cp310-cp310-linux_x86_64.whl size=4266272 sha256=d404b58dfc3890f49405feb3725d8d54bb4087efd6d29e36ee872326ebe693c9
  Stored in directory: /root/.cache/pip/wheels/96/17/12/83db63ee0ae5c4b040ee87f2e5c813aea4728b55ec6a37317c
Successfully built mpi4py


#### Importing Necessary Packages

In [2]:
# Importing pandas
import pandas as pd
# Importing Numpy
import numpy as np
# Importing MPI from mpi4py package
from mpi4py import MPI
# Importing sqrt function from the Math
from math import sqrt
# Importing Decimal, ROUND_HALF_UP functions from the decimal package
from decimal import Decimal, ROUND_HALF_UP
import time

#### Downloading the data

In [3]:
#@title Download the data
!wget -qq https://cdn.iisc.talentsprint.com/CDS/Datasets/PowerPlantData.csv

### Overview

* Load the data and perform data pre-processing
* Identify the features, target and split the data into train and test
* Implement multiple Linear Regression by estimating the coefficients on the given data
* Use MPI package to distribute the data and implement `communicator`
* Define functions for each objective and make a script (.py) file to execute using MPI command
* Use OpenMP component to predict the data and calculate the error on the predicted data
* Implement the Linear Regression from `sklearn` and compare the results

#### Exercise 1: Load data (1 point)

Write a function that takes the filename as input and loads the data in a pandas dataframe with the column names as Ambient Temperature, Exhaust Vaccum, Ambient Pressure, Relative Humidity and Energy Output respectively.

**Hint:** read_csv()


In [4]:
FILENAME = "/content/PowerPlantData.csv" # File path
# YOUR CODE HERE to Define a function to load the data
df = pd.read_csv(FILENAME)
df.head()

,AT,V,AP,RH,PE
0,8.34,40.77,1010.84,90.01,480.48
1,23.64,58.49,1011.40,74.20,445.75
2,29.74,56.90,1007.15,41.91,438.76
3,19.07,49.69,1007.22,76.79,453.09
4,11.80,40.66,1017.13,97.20,464.43


#### Exercise 2: Explore data (1 point)

Write a function that takes the data loaded using the above defined function as input and explore it.

**Hint:** You can define and check for following things in the dataset inside a function

- checking for the number of rows and columns
- summary of the dataset
- check for the null values
- check for the duplicate values

In [5]:
# YOUR CODE HERE
print(f"Number of rows: {df.shape[0]}")
print(f"Number of columns: {df.shape[1]}")
print(f"Number of null values: {df.isnull().sum().sum()}")
print(f"Number of duplicate values: {df.duplicated().sum()}")
df.describe()

Number of rows: 9568
Number of columns: 5
Number of null values: 0
Number of duplicate values: 41


,AT,V,AP,RH,PE
count,9568.000000,9568.000000,9568.000000,9568.000000,9568.000000
mean,19.651231,54.305804,1013.259078,73.308978,454.365009
std,7.452473,12.707893,5.938784,14.600269,17.066995
min,1.810000,25.360000,992.890000,25.560000,420.260000
25%,13.510000,41.740000,1009.100000,63.327500,439.750000
50%,20.345000,52.080000,1012.940000,74.975000,451.550000
75%,25.720000,66.540000,1017.260000,84.830000,468.430000
max,37.110000,81.560000,1033.300000,100.160000,495.760000


In [6]:
df.isna().sum()

,0
AT,0
V,0
AP,0
RH,0
PE,0


#### Exercise 3: Handle missing data (1 point)

After exploring the dataset if there are any null values present in the dataset then define a function that takes data loaded using the above defined function as input and handle the null values accordingly.

**Hint:**

- Drop the records containing the null values - dropna()
- Replace the null values with the mean/median/mode - fillna()

In [7]:
# Function to handle missing data

# YOUR CODE HERE
df.fillna(df.mean(), inplace=True)
df.dropna(inplace=True)
df.isnull().sum()

,0
AT,0
V,0
AP,0
RH,0
PE,0


In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9568 entries, 0 to 9567
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   AT      9568 non-null   float64
 1   V       9568 non-null   float64
 2   AP      9568 non-null   float64
 3   RH      9568 non-null   float64
 4   PE      9568 non-null   float64
dtypes: float64(5)
memory usage: 373.9 KB


#### Exercise 4: Scale the data (1 point)

Write a function that takes the data after handling the missing data as input and returns the standardized data.

**Hint:**

- standardization of the data  can be performed using the below formula

$ (x - mean(x)) / std(x) $

In [9]:
# Defining a function to standardize the data
# YOUR CODE HERE
def standardize(df):
    return (df - df.mean())/df.std()
df_scaled = standardize(df)
df_scaled

,AT,V,AP,RH,PE
0,-1.517782,-1.065149,-0.407336,1.143885,1.530146
1,0.535228,0.329260,-0.313040,0.061028,-0.504776
2,1.353748,0.204141,-1.028675,-2.150575,-0.914338
3,-0.077992,-0.363223,-1.016888,0.238422,-0.074706
4,-1.053507,-1.073805,0.651804,1.636341,0.589734
...,...,...,...,...,...
9563,-0.608017,-0.423816,-0.245686,-0.025957,0.481924
9564,1.846202,1.860591,-0.498263,-0.930735,-1.257691
9565,-0.491277,-0.862913,0.158437,0.366502,0.679381
9566,-0.268532,0.437854,0.895962,1.461687,-0.201266


#### Exercise 5: Feature selection (1 point)

Write a function that takes scaled data as input and returns the features and target variable values

**Hint:**

- Features: AmbientTemperature, ExhaustVaccum, AmbientPressure, RelativeHumidity
- Target Variable: EnergyOutput

In [10]:
# Define a function
def get_feature_target(df):
    # YOUR CODE HERE
    features = df.drop('PE', axis=1)
    target = df['PE']
    return features, target
# YOUR CODE HERE
features, target = get_feature_target(df_scaled)

#### Exercise 6: Correlation (1 point)

Calculate correlation between the variables

In [11]:
# YOUR CODE HERE
print(features.corr())
#Calculate correlation heat Map
features.corr().style.background_gradient(cmap='coolwarm')

          AT         V        AP        RH
AT  1.000000  0.844107 -0.507549 -0.542535
V   0.844107  1.000000 -0.413502 -0.312187
AP -0.507549 -0.413502  1.000000  0.099574
RH -0.542535 -0.312187  0.099574  1.000000


,AT,V,AP,RH
AT,1.000000,0.844107,-0.507549,-0.542535
V,0.844107,1.000000,-0.413502,-0.312187
AP,-0.507549,-0.413502,1.000000,0.099574
RH,-0.542535,-0.312187,0.099574,1.000000


#### Exercise 7: Estimate the coefficients (2 points)

Write a function that takes features and target as input and returns the estimated coefficient values

**Hint:**

- Calculate the estimated coefficients using the below formula

$ β = (X^T X)^{-1} X^T y $

- transpose(), np.linalg.inv()

In [12]:
# Calculating the coeffients

# YOUR CODE HERE
def calculateEstimatedCoefficients(features, target):
    X = np.array(features)
    y = np.array(target)
    coefficients = np.linalg.inv(X.transpose().dot(X)).dot(X.transpose()).dot(y)
    return coefficients

calculateEstimatedCoefficients(features, target)

array([-0.86350078, -0.17417154,  0.02160293, -0.13521023])

#### Exercise 8: Fit the data to estimate the coefficients (2 points)

Write a function named fit which takes features and targets as input and returns the intercept and coefficient values.

**Hint:**

- create a dummy column in the features dataframe which is made up of all ones
- convert the features dataframe into numpy array
- call the estimated coefficients function which is defined above
- np.ones(), np.concatenate()

In [13]:
# defining a fit function
def fit(x, y):
    temp = x.copy()
    # YOUR CODE HERE
    temp.insert(loc=0, column='DUMMY', value=1)
    return calculateEstimatedCoefficients(temp, y)
calc_coff = fit(features, target)

In [14]:
intercept=calc_coff[0]
coefficients=calc_coff[1:]
intercept, coefficients

(-1.590307746601738e-15,
 array([-0.86350078, -0.17417154,  0.02160293, -0.13521023]))

In [15]:
print(f"Intercept={intercept}")
print(f"Coefficients={coefficients}")

Intercept=-1.590307746601738e-15
Coefficients=[-0.86350078 -0.17417154  0.02160293 -0.13521023]


#### Exercise 9: Predict the data on estimated coefficients (1 point)

Write a function named predict which takes features, intercept and coefficient values as input and returns the predicted values.

**Hint:**

- Fit the intercept, coefficients values in the below equation

  $y = b_0 + b_1*x + ... + b_i*x_i$

In [21]:
 # fucntion to predict the values
 #YOUR CODE HERE
def predict(x, intercept, coefficients):
    start_time = time.perf_counter()
    predictions = intercept + np.dot(x, coefficients)
    end_time = time.perf_counter()
    print('Elapsed wall clock time = %g seconds.' % (end_time-start_time) )
    return predictions
predict(features, calc_coff[0], calc_coff[1:])

Elapsed wall clock time = 0.00900642 seconds.


array([ 1.33266027, -0.53453122, -0.93596031, ...,  0.52838115,
       -0.02266325, -0.41153611])

#### Exercise 10: Root mean squared error (1 point)

Write a function to calculate the RMSE error.

**Hint:**

- [How to calculate the RSME error](https://towardsdatascience.com/what-does-rmse-really-mean-806b65f2e48e)

In [23]:
# Define a function to calculate the error

# YOUR CODE HERE
y_pred = predict(features, calc_coff[0], calc_coff[1:])

def calculate_rmse(actual, predicted):
    start_time = time.perf_counter()
    mse = np.mean((actual - predicted) ** 2)
    rmse = np.sqrt(mse)
    end_time = time.perf_counter()
    print('Elapsed wall clock time = %g seconds.' % (end_time-start_time) )
    return rmse
# YOUR CODE HERE
rmse = calculate_rmse(target, y_pred)

Elapsed wall clock time = 0.00682473 seconds.
Elapsed wall clock time = 0.000837648 seconds.


In [18]:
round(rmse, 4)

0.267

#### Exercise 11: Split the data into train and test (1 point)

Write a function named train_test_split which takes features and targets as input and returns the train and test sets respectively.

**Hint:**

- Shuffle the data
- Consider 70 % of data as a train set and the rest of the data as a test set

In [19]:
# YOUR CODE HERE
# shuffle the DataFrame rows
def train_test_split(features, target, random_state=None):
    df_new = features.copy()
    df_new['target'] = target

    shuffled = df_new.sample(frac=1).reset_index(drop=True)

    # Calculate the split index
    split_index = int(0.7* len(shuffled))

    train = shuffled[:split_index]
    test = shuffled[split_index:]

    X_train = train.drop('target', axis=1)
    y_train = train['target']
    X_test = test.drop('target', axis=1)
    y_test = test['target']
    return X_train, X_test, y_train, y_test

X_train, X_test, y_train, y_test = train_test_split(features, target, random_state=42)

X_train.shape, X_test.shape, y_train.shape, y_test.shape

((6697, 4), (2871, 4), (6697,), (2871,))

#### Exercise 12: Implement predict using OpenMP (1 point)

Get the predictions for test data and calculate the test error(RMSE) by implementing the OpenMP (pymp)

**Hints:**

* Using the pymp.Parallel implement the predict function (use from above)

* Call the predict function by passing test data as an argument

* calculate the error (RMSE) by comparing the Actual test data and predicted test data

In [20]:
!pip install pymp-pypi

  Preparing metadata (setup.py) ... done
  Created wheel for pymp-pypi: filename=pymp_pypi-0.5.0-py3-none-any.whl size=10314 sha256=97fb37a81bf805d887a63644cc564cc799ababb40297be278c033bae5d4b8ddd
  Stored in directory: /root/.cache/pip/wheels/5e/db/4b/4c02f5b91b1abcde14433d1b336ac00a09761383e7bb1013cf
Successfully built pymp-pypi


In [45]:
import pymp
# YOUR CODE HERE
#Predict Function
def predict_pymp(x, intercept, coefficients):
    start_time = time.perf_counter()
    predicted_vals = pymp.shared.array((len(features)), dtype='float')
    with pymp.Parallel(4) as p:
        for i in p.range(len(x)):
            predicted_vals[i] = intercept + np.dot(x.iloc[i], coefficients)
    end_time = time.perf_counter()
    print('Elapsed wall clock time = %g seconds.' % (end_time-start_time))
    return predicted_vals

y_pred_pymp = predict_pymp(features, calc_coff[0], calc_coff[1:])
y_pred_pymp

Elapsed wall clock time = 0.502969 seconds.


array([ 1.33266027, -0.53453122, -0.93596031, ...,  0.52838115,
       -0.02266325, -0.41153611])

In [57]:
#RMSE
def calculate_rmse_pymp(actual, predicted):
    start_time = time.perf_counter()
    mse_vals = pymp.shared.array((len(actual)), dtype='float')
    with pymp.Parallel(4) as p:
        for i in p.range(len(actual)):
            mse_vals[i] = np.mean((actual[i] - predicted[i]) ** 2)
        rmse = np.sqrt(mse_vals.sum()/len(actual))

    end_time = time.perf_counter()
    print('Elapsed wall clock time = %g seconds.' % (end_time-start_time) )
    return rmse
# YOUR CODE HERE
rmse = calculate_rmse_pymp(target, y_pred_pymp)
round(rmse,4)

Elapsed wall clock time = 0.179306 seconds.


0.267

In [56]:
rmse = calculate_rmse(target, y_pred_pymp)
round(rmse,4)

Elapsed wall clock time = 0.00513019 seconds.


0.267

#### Exercise 13: Create a communicator (1 point)

Create a comunicator and define the rank and size

In [58]:
# YOUR CODE HERE
comm = MPI.COMM_WORLD
rank = comm.Get_rank()
size = comm.Get_size()
print(f"Rank: {rank}")
print(f"Size: {size}")

Rank: 0
Size: 1


#### Exercise 14: Divide the data into slices (1 point)

Write a function named dividing_data which takes train features set, train target set, and size of workers as inputs and returns the sliced data for each worker.

![img](https://cdn.iisc.talentsprint.com/CDS/Images/MiniProject_MPI_DataSlice.JPG)

For Example, if there are 4 processes, slice the data into 4 equal parts with 25% ratio

**Hint:**

- Divide the Data equally among the workers
  - Create an empty list
  - Iterate over the size of workers
  - Append each slice of data to the list

In [68]:
np.array(features)[9567]

array([ 0.54059487, -0.23653046, -0.23558329, -0.1417082 ])

In [80]:
def dividing_data(x_train, y_train, size_of_workers):
    sliced_data=[]
    # Size of the slice
    slice_for_each_worker = int(Decimal(x_train.shape[0]/size_of_workers).quantize(Decimal('1.'), rounding = ROUND_HALF_UP))
    print('Slice of data for each worker: {}'.format(slice_for_each_worker))
    # YOUR CODE HERE
    for i in range(size_of_workers):
        start = i*slice_for_each_worker
        if i*slice_for_each_worker == 0:
            start = 0
        end = (i+1)*slice_for_each_worker
        if i == size_of_workers-1:
            end = x_train.shape[0]
        sliced_data.append((features.iloc[start:end], target.iloc[start:end]))
    return sliced_data

sliced_data = dividing_data(features, target, 4)

Slice of data for each worker: 2392


In [83]:
sliced_data

[(            AT         V        AP        RH
  0    -1.517782 -1.065149 -0.407336  1.143885
  1     0.535228  0.329260 -0.313040  0.061028
  2     1.353748  0.204141 -1.028675 -2.150575
  3    -0.077992 -0.363223 -1.016888  0.238422
  4    -1.053507 -1.073805  0.651804  1.636341
  ...        ...       ...       ...       ...
  2387  0.847875  1.183060 -0.644758  1.013750
  2388 -1.716374 -1.017148  1.362387  1.196623
  2389 -0.183997 -1.039181 -1.895856 -0.521838
  2390 -0.579839 -0.946326 -0.853555 -0.490332
  2391 -1.250757 -1.125742  1.362387  1.200733
  
  [2392 rows x 4 columns],
  0       1.530146
  1      -0.504776
  2      -0.914338
  3      -0.074706
  4       0.589734
            ...   
  2387   -1.133475
  2388    1.940880
  2389    0.331927
  2390    0.634851
  2391    1.462178
  Name: PE, Length: 2392, dtype: float64),
 (            AT         V        AP        RH
  2392 -1.521808 -1.089544  1.146855  0.333626
  2393  0.739187 -0.008326 -0.095824 -1.047171
  2394 -0.644

#### Exercise 15: Prepare the data in root worker to assign data for all the workers (1 point)

- When it is the root worker, perform the below operation:
    - Store the features and target values in separate variables
    - Split the data into train and test sets using the train_test_split function defined above
    - Divide the data among the workers using the dividing_data function above

In [ ]:
# YOUR CODE HERE

#### Exercise 16: Scatter and gather the data (1 point)

Perform the below operations:

- Send slices of the training set(the features data X and the expected target data Y) to every worker including the root worker
    - **Hint:** scatter()
    - use `barrier()` to block workers until all workers in the group reach a Barrier, to scatter from root worker.
- Every worker should get the predicted target Y(yhat) for each slice
- Get the new coefficient of each instance in a slice
    - **Hint:** fit function defined above
- Gather the new coefficient from each worker
    - **Hint:** gather()
    - Take the mean of the gathered coefficients
- Calculate the root mean square error for the test set

To know more about `scatter`, `gather` and `barrier` click [here](https://nyu-cds.github.io/python-mpi/05-collectives/)

In [ ]:
# YOUR CODE HERE

#### Exercise 17: Make a script and execute everything in one place (1 point)

Write a script(.py) file which contains the code of all the above exercises in it so that you can run the code on multiple processes using MPI.

**Hint:**

- magic commands
- put MPI related code under main function
- !mpirun --allow-run-as-root -np 4 python filename.py

In [ ]:
# YOUR CODE HERE for scipt(.py)

In [ ]:
# YOUR CODE HERE for MPI command

#### Exercise 18: Use Sklearn to compare (1 point)

Apply the Linear regression on the given data using sklearn package and compare with the above results

**Hint:**
* Split the data into train and test
* Fit the train data and predict the test data using `sklearn Linear Regression`
* Compare the coefficients and intercept with above estimated coefficients
* calculate loss (RMSE) on test data and predictions and compare

In [ ]:
# YOUR CODE HERE